In [58]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
from shutil import copy

In [ ]:
''' Custom function to retrieve the seasonality data frame. '''

def seasonality_df(m, yearly_start=0,  name='yearly'):
    from fbprophet.plot import seasonality_plot_df
    """Plot the yearly component of the forecast.
    Parameters
    ----------
    m: Prophet model.
    ax: Optional matplotlib Axes to plot on. One will be created if
        this is not provided.
    uncertainty: Optional boolean to plot uncertainty intervals, which will
        only be done if m.uncertainty_samples > 0.
    yearly_start: Optional int specifying the start day of the yearly
        seasonality plot. 0 (default) starts the year on Jan 1. 1 shifts
        by 1 day to Jan 2, and so on.
    figsize: Optional tuple width, height in inches.
    name: Name of seasonality component if previously changed from default 'yearly'.
    Returns
    -------
    a dataframe
    """
    # Compute yearly seasonality for a Jan 1 - Dec 31 sequence of dates.
    days = (pd.date_range(start='2017-01-01', periods=365) +
            pd.Timedelta(days=yearly_start))
    df_y = seasonality_plot_df(m, days)
    seas = m.predict_seasonal_components(df_y)
    return seas


In [53]:


import os
from os.path import join, getsize
in_directory = '/Users/angelocortez/Desktop/githubprojects/redtide/redtide/data/full_history'
out_directory = '/Users/angelocortez/Desktop/githubprojects/redtide/redtide/data/yearly_seasonality'
for root, dirs, files in os.walk(in_directory):
    for file in files:
        if file.endswith('AAPL.csv'):
            data = pd.read_csv(join(in_directory, file))
            data['daily_return']=data['adjclose']/data['adjclose'].shift(1)-1
            values = {'daily_return' : 0 }
            data.fillna(value = values, inplace = True)
            
            # create seasonality df
            df = data.reset_index().rename(columns={'date':'ds', 'daily_return':'y'})
            m = Prophet(daily_seasonality=False).fit(df)
            seas_df = seasonality_df(m)

            # create new df with the first 30 days of the new year appended to the end
            df1 = seas_df['yearly'][:]
            df2 = seas_df['yearly'][:31]
            df3 = pd.concat([df1,df2])
            df3.index = [int(x) for x in range(0,len(df3))]
            df3 = df3.reset_index().rename(columns={'yearly':'cum_ret'})
            #second error
            
            
            df3.to_csv(join(out_directory, file), header = True ,index_label = "idx")
            

/anaconda3/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [85]:
# seasonality screen
in_directory = '/Users/angelocortez/Desktop/githubprojects/redtide/redtide/data/yearly_seasonality'
out_directory = '/Users/angelocortez/Desktop/githubprojects/redtide/redtide/data/ret'
for root, dirs, files in os.walk(in_directory):
    for file in files:
        if file.endswith('.csv'):
            path = join(in_directory, file)
            data = pd.read_csv(path, index_col = 'idx')
            
            # Story intervals where there are positive gains on average
            count = 0 
            ret_int = []
            interval_arr = []
            interval = []
            for i,j in zip(data['index'], data['cum_ret']):
                if j < 0:
                    count = 0
                    continue
                else:
                    if count == 0 and j >= 0: 
                        interval = [i,i]
                        interval_arr.append(interval)
                    elif count > 0 and j >= 0:
                        interval_arr[-1][1] += 1 
                    count += 1
                    
            # store to array if seasonality is detected
            for tup in interval_arr:
                if (tup[1] - tup[0]) >= 31:
                    ret_int.append(tup)
                    
            # quit if no seasonality > 31 days
            if ret_int == []:
                print("not finding seasonality for " +  file)
                continue
            file_name, extension = os.path.splitext(file)
            file_name += ','
            for tup in ret_int:
                file_name += "-".join([str(x) for x in tup]) 
                file_name += ','
            file_name = file_name[:len(file_name)-1]
            print("copying " + file_name)
            copy(path, join(out_directory, file_name))
            #data.to_csv(join(out_directory, file_name), header = True ,index_label = "idx")
            

copying RIV,234-324
copying ANTE,159-241
copying CSCO,20-103,197-282
copying PRI,115-146,198-236
copying SLIM,60-104,243-319
copying BIOS,52-93,247-320
copying NZF,35-68,255-288,298-349
copying HLNE,58-92,192-226,250-305
copying UNT,149-219,256-345
not finding seasonality for SPXB.csv
copying HUBS,65-102,239-290,308-357
copying GPL,257-315
copying PSM,14-59,241-276
copying SBGI,186-239
copying UFCS,270-319
copying GFED,308-340
not finding seasonality for AIRT.csv
copying SITC,255-326
copying NXN,46-102,122-168,239-346
copying BNY,46-77,127-170,256-288,301-351
copying ISSC,197-238,299-332
copying EFT,120-178,254-286,300-342
copying TRNS,216-286
copying NTZ,159-224,308-343
copying AEIS,232-288
copying TROW,222-289
copying ISRG,213-281,307-363
copying EGP,194-237,266-314
copying NVR,152-205,273-319
copying FEIM,53-107,297-328
copying WOR,189-282,303-354
not finding seasonality for PRGO.csv
copying LECO,198-242
copying RPT,258-290
copying COHN,165-221,299-354
copying NBW,135-170,199-236,30

copying GENC,200-236
copying CYAN,75-106,242-278
copying MSI,262-300
copying BC,125-189,212-308
copying AX,119-188,263-306
copying FCX,198-232
copying HNP,194-235,259-307
copying WASH,192-262,296-338
copying HLX,0-48,123-165,196-228,294-345,364-395
copying B,0-36,257-291,347-395
copying ADM,45-106,154-215
copying CP,215-278,302-345
copying ULBI,81-132,196-239
copying ASUR,25-100,180-216
not finding seasonality for VFH.csv
copying HAIN,305-345
copying HSTM,48-81,302-352
copying FCO,46-95,201-239,303-351
not finding seasonality for STXS.csv
copying INSG,39-90,192-228
copying VHC,160-218,242-283
copying CSF,110-193
copying TRX,121-163
copying FMS,196-246
copying OEF,202-238
copying CSQ,203-234
copying LH,193-229,236-286,310-364
copying VHT,259-290
copying BOTJ,68-109,222-280,308-341
copying LTM,120-181,200-276
copying GGG,111-170,195-264
copying ZEUS,165-318
copying GEO,186-246
copying IHG,127-177,198-240
copying PFN,36-68,172-231,252-284,298-341
copying MTSC,195-241
copying NMZ,301-349
c

copying R,251-301
copying AEY,175-207
copying MSN,89-142,249-340
copying AFB,25-67,258-349
copying MGIC,45-95,233-281
copying CW,201-246,254-312
copying VGK,122-178
copying E,5-60,184-237,278-340
copying CUTR,113-145
copying CYBE,50-90,161-225,246-280
copying MPB,72-104,155-209,241-293,341-380
not finding seasonality for CERN.csv
copying MET,4-65
copying OTEL,200-236
not finding seasonality for CIX.csv
copying MDP,168-236
copying CHK,1-43,169-221,290-345
copying VPU,198-250,350-388
copying WVFC,136-186
copying BJRI,54-89,336-368
copying DLR,114-146,244-277,286-324
copying STAA,50-95,174-207
copying ACGL,118-168,313-349
copying PFBI,190-250,347-389
copying TCBI,195-237,335-387
copying XRAY,1-52,137-178
not finding seasonality for VV.csv
not finding seasonality for BFIN.csv
copying ACHC,39-92,177-231,254-293,304-347
copying TGI,200-234,243-283
copying INGR,245-278
copying HCKT,245-278
copying LAMR,6-63,198-249
copying JBLU,4-67
copying JOF,128-166,191-223,228-260,288-350
copying SBT,164-

copying EVY,252-287,295-347
copying BSD,26-79,128-166,254-351
copying CONN,265-312
copying KYN,249-339
copying PLX,283-350
copying PBS,199-234
copying BRID,97-131,265-296
copying DXCM,168-207,245-320
copying DSPG,241-295
copying NKX,44-89,302-347
copying NHC,188-245
copying EXR,259-290
copying HIBB,183-238,339-388
copying KWR,118-163,185-235,256-299
copying WELL,244-343
copying BRKL,195-239,259-290
copying HRTX,49-105,194-234,256-287,306-349
copying DRQ,120-172,181-219,253-292,301-335
copying TTM,8-45,112-189
copying VMM,49-102,255-348
copying HOLX,76-109,243-289
copying EXPD,190-237
copying DSU,20-66,241-291,303-342
copying SSYS,54-93,194-226
copying TVE,45-108,247-282,311-352
copying KZIA,127-222
copying TXN,229-302
copying VERU,175-209
copying XLF,1-60,202-241
copying MTX,196-244,318-349
copying ASRT,93-130,171-229,256-301
not finding seasonality for JSM.csv
copying FFA,255-287
not finding seasonality for FEZ.csv
copying BVSN,71-130,188-219
copying VCF,48-84,200-231,265-312
copying 

copying RWT,188-228,260-294
not finding seasonality for NDAQ.csv
copying CVCO,191-235,305-358
copying URI,192-233
copying BQH,49-81,258-290,297-343
copying COLB,49-95,130-206
copying WHR,199-307
not finding seasonality for USM.csv
copying RVP,318-353
copying PMO,254-349
copying ICF,253-324
copying ANIK,128-179,263-327
copying ETB,203-234,299-346
copying EWY,241-288
copying WIA,112-170,227-283,308-344
copying LYTS,195-231
copying NEW,211-290
copying KBAL,266-318
copying PMX,28-64,257-349
copying PNC,258-290
not finding seasonality for KMX.csv
copying NAII,134-233
copying FELE,226-292
copying EAT,122-176,196-246
copying BGR,195-232,299-340
copying NPV,48-81,231-353
copying SLAB,245-276,314-348
not finding seasonality for CMCO.csv
copying RLI,91-137,193-254
copying PWV,203-234
copying ENLC,293-343
copying ELS,93-159,193-243,271-327
copying WSO,190-222,231-298
not finding seasonality for IYH.csv
copying LEU,205-282
copying WPC,10-60,200-264,295-328
copying PWC,197-229
copying ROG,199-281,3

copying MMP,126-168,239-281
copying VXF,196-228
copying SMED,296-352
copying HPI,18-66,247-283,297-329
not finding seasonality for SCHL.csv
copying CENX,249-321
copying SGA,239-272
copying MRLN,32-63,269-334
copying HQL,47-103,251-283
copying TCX,186-305
copying CCB,133-180
copying HTLD,119-174
copying SASR,160-211,350-388
copying SFE,157-219,251-287
copying STKL,182-220,258-352
copying CRNT,47-102
copying TAP,9-54,258-310
copying FMBH,257-294,319-378
copying CAJ,2-59,274-310
copying EFOI,61-112,233-294,308-350
copying CCU,124-180,212-288
copying TCO,195-230,258-290
copying SHY,63-123,270-305
copying AVA,30-61
copying MAR,125-177,194-282
copying BMRA,303-355
copying MBI,251-319
copying ATAX,126-165,264-351
copying AMNB,109-141,192-225,272-303,354-392
copying CMI,243-310
copying CNR,184-237,260-291
copying TOL,122-212,248-304
copying CHSCP,81-130,257-316
copying CLM,205-288,301-353
copying AWR,238-293
copying ATI,193-229,249-282
not finding seasonality for FRO.csv
copying KMPR,1-34,154-

copying JPM,195-287
copying CNBKA,44-77,107-147,285-350
copying ADRU,205-238
copying PATK,259-351
copying ABT,48-79,113-169,208-268,294-348
copying FR,13-59,163-238,265-322
copying MTC,42-93,231-387
copying FFIC,303-338
copying OMC,126-243
copying GV,244-348
copying RELL,59-104,238-271
copying OFIX,161-208,256-297
copying HALL,300-349
not finding seasonality for BOKF.csv
not finding seasonality for VUG.csv
copying COF,9-58
copying AMOV,114-166,198-281
copying TSCO,114-184,303-356
copying SHI,145-198,253-341
copying VIRC,46-132,188-229,294-348
copying MLHR,199-314
copying ASGN,158-218
copying PH,52-86,120-221,244-283
copying VWO,117-175,294-330
copying TSBK,123-187,233-288,328-360
copying SIM,126-163,199-241,264-313,354-386
copying CIVB,111-143,307-353
copying MEOH,253-289
copying HSKA,124-186,256-289
copying MLR,24-67,160-213,226-283
copying HST,200-235,244-280
copying ATLO,0-61,202-235,362-395
copying TARO,56-87,293-358
copying AYI,114-146,169-230
copying PCTI,104-146,257-292
not find

copying WMS,158-235,259-310
copying CDZI,197-229,260-292
copying EPP,116-147
copying EQT,140-213,243-296
copying TILE,181-217,243-301
copying ICAD,191-227
copying BTI,309-340
copying GHC,209-302
copying IGT,134-181
copying EQC,254-342
copying FNLC,197-239,275-310,362-393
copying LBTYB,27-69,261-298
copying WBK,208-240
copying PLAB,187-219,246-292
copying OPHC,111-172,264-299
copying SBRA,12-67
copying MSVB,261-304
copying PEI,198-240
copying GES,251-283
copying LUB,199-301
copying FLXS,102-134,170-251,265-298
not finding seasonality for MVIS.csv
copying RVSB,117-172
copying EBIX,194-305
copying IJS,194-226,252-284
copying KSS,84-140,254-286,313-379
copying USLM,10-56,181-239,257-306
copying PFE,47-78,219-264
copying CSWC,195-233,255-292
copying OGE,9-68,116-150
copying WSBC,191-235,290-321
copying AIV,23-57,249-327
copying CRI,190-234,315-382
copying DYNT,116-168,261-354
copying SWN,154-226,299-348
not finding seasonality for DUK.csv
copying FMO,240-289,297-337
copying TPL,248-303
copy

copying CTHR,260-309,333-383
copying DJCO,11-45,79-113,300-352
copying BBQ,233-289
copying VVUS,83-116,188-222,263-312
copying IDRA,118-181
copying NXR,248-347
copying EHC,67-98
copying RJF,156-236
copying BMI,74-131,247-308
copying IONS,66-100
copying HBAN,2-59,199-237
copying UNH,194-231,255-290
copying GSK,171-214
copying KGC,143-211,259-311
copying PRU,5-63
copying WTS,64-97,252-299
copying EGBN,57-89,186-236
not finding seasonality for MORN.csv
not finding seasonality for BOE.csv
copying PSB,27-60
copying BGCP,193-225
copying GPC,157-229,255-292
copying MOSY,175-226
copying LCI,187-235
copying KEX,182-228,242-345
copying FISV,198-304
copying SRCE,234-296
copying LNN,117-170,192-240
copying IRS,146-194,207-245
copying SPTN,195-266
copying NWN,25-94
copying RDWR,58-108,251-284
copying UHAL,10-64,122-156,193-227
copying IDSA,253-329
copying RDI,155-192,217-344
copying RGR,248-314
copying NTWK,196-237
copying CASS,0-40,195-312,338-395
copying BURG,168-213
not finding seasonality for R

copying SNH,292-333
copying TV,32-65,193-281,352-391
copying WM,22-53,167-289,316-357
copying CHT,265-308
copying DOV,244-292
copying FTI,130-178,286-341
copying CKX,177-230,251-299
copying CIG,200-269
not finding seasonality for VGT.csv
copying AGN,39-106,297-329
copying MQY,24-90,240-316
copying MERC,131-205,246-284
copying IBKC,196-238,260-293
copying M,155-195
copying NWPX,192-242,254-339
copying AEF,197-286,300-337
copying OSTK,0-33,189-234,344-395
copying FLO,182-240,325-385
copying AIR,248-279
copying MT,116-147,251-287
copying CRM,149-227,306-353
copying CPE,176-234,310-344
copying CLWT,103-146,247-307
copying NX,194-225,255-319
copying NNY,52-98,231-322
copying USNA,31-72,159-190,275-365
copying PGR,302-340
copying ELGX,205-237,261-350
copying WAT,49-100,139-213,261-297,321-368
copying NNN,119-174,241-280,301-346
copying MTSL,55-142
copying NAVB,194-229
copying IMAX,139-179,196-241
copying IMOS,162-221,268-308
copying IEX,196-282
copying HWBK,167-233,254-302
copying BBGI,196-2

In [ ]:
#l